## Final Project Submission - Phase 2

* Student name: Magali Solimano
* Student pace: self paced
* Scheduled project review date/time: 
* Instructor name: Claude Fried
* Blog post URL:


### Goal of analysis: 

To inform King County, WA single-family home owners of features that affect housing prices.The county has commissioned analysis of its house sales to assist home buyers and sellers to make informed decision-making in the residential real estate market. The analysis will be publicly available for all county residents.

In [ ]:
# Import libraries and functions
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
%matplotlib inline
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Import dataset for King County House Sales
df = pd.read_csv('data/kc_house_data.csv')

# Column names and descriptions for King County Data Set
* **id** - unique identified for a house
* **date** - house was sold
* **price** -  prediction target
* **bedrooms** -  number of Bedrooms/House
* **bathrooms** -  number of bathrooms/bedrooms
* **sqft_living** - square footage of the home
* **sqft_lot** - square footage of the lot
* **floors** -  total floors (levels) in house
* **waterfront** - House which has a view to a waterfront
* **view** - Has been viewed
* **condition** - How good the condition is (Overall)
* **grade** - overall grade given to the housing unit, based on King County grading system. Classification by construction quality which refers to the types of materials used and the quality of workmanship.
* **sqft_above** - square footage of house apart from basement
* **sqft_basement** - square footage of the basement
* **yr_built** - Built Year
* **yr_renovated** - Year when house was renovated
* **zipcode** - zip
* **lat** - Latitude coordinate
* **long** - Longitude coordinate
* **sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbors
* **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbors

# Understanding the dataset

In [ ]:
df.info()

In [ ]:
df.duplicated().value_counts()

In [ ]:
df.head()

- Three variables have missing values: waterfront, view, yr_renovated.
- Some variables will require changing dtype classifications: date, sqft_basement.
- There are no duplicate entries.

In [ ]:
df.describe()

In [ ]:
# Examine distributions to better understand data and what preparation the data may need

fig, axes = plt.subplots(5,5, figsize=(15,15))
fig.set_tight_layout(True)

for index, col in enumerate(df.columns):
    ax = axes[index//5][index%5]
    ax.hist(df[col])
    #ax.set_xlabel(col)
    
    ax.set_title(col)
    
fig.tight_layout()   

- Data for dependent variable (price) and many independent variables (bedrooms, bathrooms, sqft_living, sqft_lot, other sqft variables) are highly skewed, with large outliers. Many variables are positively skewed, with large outliers in right tail. Since this analysis is focused on prices for single-family homes, outlier analysis will start with removing outliers for price, number of bedrooms, and number of bathrooms that exceed what would be expected to seen for single-family homes in a normal distribution.
- Additional data could be extracted from date, such as quarter sold, month sold, day sold.
- There appear to be categorical variables (waterfront) and other variables (eg, zipcode, year renovated) can perhaps be treated as categorical as well.
- id variable may not be useful feature for this home price analysis, but could shed light on records in the datasets.

In [ ]:
# Do some home ids appear more than once in the dataset?
df.id.value_counts()

In [ ]:
display(df.groupby('id')['id', 'date', 'price'].filter(lambda x: len(x) > 1).nunique())
df.groupby('id')['id', 'date', 'price'].filter(lambda x: len(x) > 1)

- 176 homes sold more than once, and this accounts for a small share of the dataset (0.8%). Some homes may have been flipped, or deals may have fallen through.

## Data preparation

In [ ]:
# DATA TYPE CONVERSIONS
# Convert date to date dtype
df_pp = df  #df for preprocessing of data
df_pp.date = pd.to_datetime(df['date'])
df_pp.date

In [ ]:
# Extract year, quarter, month, day, day of week from date
df_pp['yr_sold'] = pd.DatetimeIndex(df.date).year
df_pp['quarter_sold'] = pd.DatetimeIndex(df.date).quarter
df_pp['month_sold'] = pd.DatetimeIndex(df.date).month
df_pp['day_of_week_sold'] = pd.DatetimeIndex(df.date).dayofweek  #legend: 0-Monday, 6-Sunday

In [ ]:
# Convert sqft_basement dtype from object to float
df_pp.sqft_basement.value_counts(dropna=False)

In [ ]:
# '?' value is probably a NaN, which we can replace with NaN and then replace with the mean
df_pp.sqft_basement = df.sqft_basement.replace('?', np.nan)

# Convert sqft_basement data type to float. 
df_pp['sqft_basement'] = df['sqft_basement'].astype(float)

# Replace NaN with mean values.
df_pp.sqft_basement = df.sqft_basement.fillna(df.sqft_basement.mean())
df_pp.sqft_basement.value_counts(dropna=False)

In [ ]:
# MISSING VALUES: Check for missing values in target variable (price) and dependent variables
df_pp.isna().sum()

In [ ]:
df_pp.waterfront.value_counts(dropna=False, normalize = True)

- Very few homes have waterfront views. 11% of waterfront values are null, which is somewhat high but there is comfort making an assumption that NaN value means there is no waterfront view given that this matches the most common value.

In [ ]:
# waterfront variable: replace "NaN" with mode
df_pp.waterfront = df.waterfront.fillna(df.waterfront.mode()[0])

# check new value counts
df_pp.waterfront.value_counts(dropna=False, normalize = True)

In [ ]:
df_pp.view.value_counts(dropna=False, normalize=True)

In [ ]:
plt.scatter(df_pp.view,df_pp.price)

In [ ]:
# view variable: fill NaN with the mean value. Round value since number of views should be whole integers.
df_pp.view = df.view.fillna(df.view.mean().round())
df_pp.view.value_counts(dropna=False)

In [ ]:
df_pp.yr_renovated.value_counts(dropna=False, normalize=True)

- The majority of homes have not been renovated. This variable presents data challenges: There are few data points for most years, and 18% of values are NaN, which is high. NaN is assumed to mean that the home was not renovated.

In [ ]:
# Replace yr_renovated null values with 0.
df_pp.yr_renovated = df.yr_renovated.fillna(0)
df_pp.yr_renovated.value_counts(dropna=False, normalize=True)

In [ ]:
#Is bathrooms variable really bathrooms/bedrooms? or just bathrooms/house?
df_pp.bathrooms.nunique()

In [ ]:
df_pp.bathrooms.value_counts(ascending=False)

In [ ]:
bath_test = df.bathrooms * df.bedrooms
bath_test.value_counts(ascending=False)

- Given the values in the bathrooms variable, which are high, it is unlikely that this variable captures the number of bathrooms to bedrooms. It is more likely that this variable captures the number of bathrooms per home.

# Outlier Analysis

In [ ]:
# Plot distributions vs dependent variable (price)
fig, axes = plt.subplots(5,5, figsize=(12,12))
fig.set_tight_layout(True)

for index, col in enumerate(df_pp.columns):
    ax = axes[index//5][index%5]
    ax.hist(df_pp[col])
    #ax.set_xlabel(col)
    ax.set_ylabel('Price')
    
    ax.set_title(col)
    
fig.tight_layout()   

In [ ]:
# Explore distribution of price
df_pp.price.describe()

In [ ]:
sns.distplot(df_pp.price)
plt.title('Distribution of Price')

In [ ]:
display(df_pp.price.quantile([0.50, 0.068, 0.95, 0.997]))

- As noted previously, price is positively skewed, with large outliers in the right tail. Remove price outliers. Criteria: remove outliers that are not consistent with focus on single-family home prices (eg, outliers are those that are more than 3 std above mean in normal distribution).

In [ ]:
# Set max price threshold at mean + three standard deviations (as would be found in normally distributed data).
df_pp[(np.abs(stats.zscore(df.price)) < 3)].describe()

- Price max is reduced from $7.7 M to $1.6 M.

In [ ]:
df_pp[(np.abs(stats.zscore(df.price)) < 3)].shape

In [ ]:
# How much data would be dropped if this max price threshold is applied?
print('No. rows dropped due to setting max price threshold:', (21597-21191))
print('Percent of original dataset dropped due to setting max price threshold:', (100*round(((21597-21191)/21597),3)), '%')

In [ ]:
df_pp = df_pp[(np.abs(stats.zscore(df.price)) < 3)]

In [ ]:
# Remove outliers for bedrooms and bathrooms in accordance with goal of focusing on single-family homes..

In [ ]:
# Address outliers for bedrooms
display(df_pp.bedrooms.value_counts())
df_pp.boxplot(column = 'bedrooms')

- Bedrooms variable has large outlier, '33'. Assume original entry was typo. 

In [ ]:
# Replace '33' outlier with mean value.
df_pp.bedrooms.mean().round(0)  #round mean to whole number since 3.3 bedrooms would not be realistic.

In [ ]:
df_pp.bedrooms = df_pp.bedrooms.replace([33],3)
df_pp.boxplot(column = 'bedrooms')

- Even when the '33' outlier is excluded, remaining bedroom outliers are still too large for single-family home sale analysis.

In [ ]:
# Set maximum number of bedrooms to mean + three std (as would be found for 99.7% of data in a normal distribution)
# Number of rows that would be dropped by applying max bedrooms threshold
df_pp[(np.abs(stats.zscore(df_pp.bedrooms)) < 3)].shape

In [ ]:
# How much data would be dropped if this max bedroom threshold is applied?
print('No. rows dropped due to setting max bedroom threshold:', (21191-21140))
print('Percent of original dataset dropped due to setting max price threshold:', (100*round(((21191-21140)/21597),3)), '%')

In [ ]:
# Number of rows that would be dropped by applying max bathrooms threshold
df_pp[(np.abs(stats.zscore(df_pp.bathrooms)) < 3)].shape

In [ ]:
df_pp[(np.abs(stats.zscore(df_pp.bedrooms)) < 3) & (np.abs(stats.zscore(df_pp.bathrooms)) < 3)].shape

In [ ]:
# How much data would be dropped if max bedroom and bathroom thresholds are applied (accounting for overlap)?
print('No. rows dropped due to setting max bedroom and bathroom thresholds:', (21191-21041))
print('Percent of original dataset dropped due to setting max price threshold:', (100*round(((21191-21041)/21597),4)), '%')

In [ ]:
# Set max bedroom and bathroom thresholds in df
df_pp = df_pp[(np.abs(stats.zscore(df_pp.bedrooms)) < 3) & 
              (np.abs(stats.zscore(df_pp.bathrooms)) < 3)]

In [ ]:
df_pp.shape

In [ ]:
# Recap of impact of outlier removal on dataset
print('How much data is lost by setting max thresholds for price, bedrooms, and bathrooms\nin accordance with focus on single-family homes?')
print('--------------------------------------------------------------------------------------------')
print('Total rows dropped due to new max thresholds for bedrooms and bathrooms (accounting for overlap):',21191 - 21041)
print('Percentage of original dataset:',(100*round(((21191 - 21041)/21597),4)),'%')

print('Total rows dropped due to new max thresholds for price, bedrooms, and bathrooms:', 21597 - 21041)
print('Percentage of original dataset:',(100*round(((21597 - 21041)/21597),3)),'%')

- Removing outliers for price, bedrooms, and bathrooms that were higher than 3 std from the mean results in dropping 2.6% of the dataset, which is less than the 5% maximum that could be dropped.

In [ ]:
# Distributions: What do distributions of price, bedrooms, bathrooms look like now? Did distributions of other variables change?
distributions_data = df_pp.copy() 

fig, axes = plt.subplots(7,4, figsize=(12,12))
fig.set_tight_layout(True)

for index, col in enumerate(distributions_data.columns):
    ax = axes[index//4][index%4]
    ax.hist(df_pp[col])
    
    ax.set_title(col)
    
fig.tight_layout()    

In [ ]:
df_pp.describe()

In [ ]:
df_pp.boxplot(column = 'price')

In [ ]:
fig, ax = plt.subplots(figsize = (8,5))

df_pp.price.hist(bins=50, facecolor='lightblue')

#add vertical line representing mean price
plt.axvline(df_pp.price.mean(), color='black', linestyle='dashed', linewidth=2)
min_ylim, max_ylim = plt.ylim()
plt.text(df_pp.price.mean()*1.1, max_ylim*0.9, 'Mean: ${:,.0f}'.format(df_pp.price.mean()))

#format xticks
ax.xaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
plt.xticks(rotation=30)
plt.xlabel('Price')

#format yticks
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

#remove grid line
plt.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

#title
plt.title('Distribution of Single-Family Home Prices', fontsize=14)

plt.show;

In [ ]:
# Mean vs median values for price, bedrooms, bathrooms, sqft_living
print('Price mean: ', df_pp.price.mean().round(2))
print('Price median: ', df_pp.price.median().round(2))
print('-----------------------')
print('Bedrooms mean: ', df_pp.bedrooms.mean().round(2))
print('Bedrooms median: ', df_pp.bedrooms.median().round(2))
print('-----------------------')
print('Bathrooms mean: ', df_pp.bathrooms.mean().round(2))
print('Bathrooms median: ', df_pp.bathrooms.median().round(2)) 
print('-----------------------')
print('sqft_living mean: ', df_pp.sqft_living.mean().round(2))
print('sqft_living median: ', df_pp.sqft_living.median().round(2)) 

- Removing outliers improved the distribution of bedrooms most notably. Price, bathroom distributions are also less skewed, but remain skewed--price is still positively skewed (mean>median, and bathrooms is negatively skewed (median>mean). Maximum price is now $1.6M (down from $7.7M), max bedrooms is 6 (down from 33), and max bathrooms is 4.25 (down from 7.75). These values are more representative of values that would be expected for single-family homes.
- Distribution of sqft_living also improved as a result of removing outliers of price, bedrooms, and bathrooms. Max sqft_living is now 7350 (down from 13,540). Nonetheless, all sqft variables remain positively skewed.
- Price and sqft variables may need to be log transformed. 

In [ ]:
# Sqft_lot appears to have one very large outlier that can be replaced with mean
df_pp.boxplot(column = 'sqft_lot')

In [ ]:
df_pp.sqft_lot.max()

In [ ]:
df_pp.sqft_lot.replace(df_pp.sqft_lot.max(), df_pp.sqft_lot.mean(), inplace=True)
#df_pp.sqft_lot.replace([233481], df_pp.sqft_lot.mean(), inplace=True)

In [ ]:
df_pp.boxplot(column = 'sqft_lot')

# Exploratory Data Analysis

In [ ]:
# Scatterplots of price and features
scatterplot_data = df_pp.drop(['price'], axis =1)

fig, axes = plt.subplots(6,4, figsize=(12,12))
fig.set_tight_layout(True)

for index, col in enumerate(scatterplot_data):
    ax = axes[index//4][index%4]
    ax.scatter(df_pp[col], df_pp['price'])
    #ax.set_xlabel(col)
    ax.set_ylabel('Price')
    
    ax.set_title(col)
    
fig.tight_layout()    
plt.show();

- Several variables have clear linear relationship with price, in particular: sqft_living, grade. 
- bedrooms, bathrooms, sqft_above, sqft_living15 also have linear relationship with price.
- Latitude has a stronger linear relationship with price than longitude, suggesting that the North-South location of the home is important for its price.
- Date-extracted variables (quarter_sold, month_sold, day of week sold) do not appear to show linear relationship with price, nor are they home features, but they could provide useful insights into market trends.
- Zipcode can be treated as categorical. Waterfont is boolean variable, and can be treated as categorical.
- Other variables that could be treated as categorical (not continuous): bedrooms, bathrooms, floors, view, condition, grade, yr_renovated.

In [ ]:
# Plot sales by quarter, month, and day of week
fig, axes = plt.subplots(1,2, figsize=(15,5))
fig.suptitle('Home Sales by Time of Year', fontsize=16, y=1)

sns.countplot(
    df_pp['quarter_sold'], color='navy', ax=axes[0]
    ).set(
    title='Quarter', 
    xlabel=None, 
    ylabel='Count');

month_name = df_pp['date'].dt.month_name().str[:3] 
month_order = ['Jan','Feb','Mar','Apr','May','Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
sns.countplot(
    month_name, color='navy', order=month_order, ax=axes[1]
    ).set(
    title='Month', 
    xlabel=None, 
    ylabel='Count');

axes[1].tick_params(axis="x", rotation=30)

sns.despine(ax=axes[0])
sns.despine(ax=axes[1])

fig.show();

- Number of home sales is higher in Q2 and Q3, and the months of spring/summer months of April, May, June, July in particular.
- Q1 is a slower period for home sales. By month, number of home sales in Jan and Feb are particularly lower than other months. 

In [ ]:
# Barplots of average number of bedrooms, bathrooms, sqft_above: for presentation slides
fig, ax = plt.subplots(figsize=(5,5))

data=df_pp[['bedrooms', 'bathrooms', 'floors']]
data = data.rename(columns={'bedrooms': 'Bedrooms', 'bathrooms': 'Bathrooms', 'floors': 'Floors'})
data.mean().plot(kind='bar', color='darkblue')

plt.xticks(rotation=0)

sns.despine()

plt.title('Home Features \n(Average Values)')

plt.show;

In [ ]:
# Barplot of average square footage: for presentation slides
fig, ax = plt.subplots(figsize=(5,5))

data=df_pp[['sqft_above']]
data = data.rename(columns={'sqft_above': 'Square Feet'})
data.mean().plot(kind='bar', color='mediumblue')

#format ticks
plt.xticks(rotation=0)
ax.set_ylim([0, 2000])
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

sns.despine()

plt.title('Average Home Square Footage \n(Mean Home Price per Square Foot: $309)')

plt.show;

In [ ]:
# Scatterplot of number of bedrooms vs. bathrooms, and how this relates to price

data = df_pp.rename(columns={'price': 'Price', 'bathrooms': 'Bathrooms', 'bedrooms': 'Bedrooms'})
fig = plt.gcf()
sns.scatterplot(data.Bedrooms, data.Bathrooms, hue = data.Price,  \
                          palette="YlOrRd", legend="brief").legend(loc='upper left',   \
                                                                   fontsize = 12, frameon = False)


fig.set_size_inches(10, 5)

plt.xlabel('Bedrooms',size=14)
plt.ylabel('Bathrooms',size=14)
sns.despine()

fig.suptitle('Price by Number of Bedrooms and Bathrooms', fontsize = 15)
fig.subplots_adjust(top=0.95)

- Unsurprisingly, homes with more bedrooms and bathrooms sell for higher prices. What is the relationship between bedrooms and bathrooms that matters for price?

In [ ]:
#bathrooms/bedroom vs. price
sns.scatterplot(df_pp.bathrooms, df_pp.price)

In [ ]:
# Create new variable: age of house when sold
df_pp['age'] = df_pp.yr_sold - df_pp.yr_built

In [ ]:
df_pp.head()

In [ ]:
# Boxplots of price and variables that could be treated as categoricals
# floors, view, condition, grade, waterfront, bedrooms, bathrooms, zipcode, yr_built
fig, axes = plt.subplots(4,2, figsize = (15,20))

sns.boxplot(data = df_pp, x='floors', y = 'price', ax = axes[0,0])

sns.boxplot(data = df_pp, x= 'view', y = 'price', ax = axes[0,1])

sns.boxplot(data = df_pp, x= 'condition', y = 'price', ax = axes[1,0])

sns.boxplot(data = df_pp, x= 'grade', y = 'price', ax = axes[1,1])

sns.boxplot(data = df_pp, x= 'waterfront', y = 'price', ax = axes[2,0])

sns.boxplot(data = df_pp, x= 'bedrooms', y = 'price', ax = axes[2,1])

sns.boxplot(data = df_pp, x= 'bathrooms', y = 'price', ax = axes[3,0])

- grade, bathrooms, views have clearest positive relationships with price.
- floors: more floors are associated with a higher price, up to a certain number of floors. More than 3 floors is related with lower price compared to 2 and 2.5 floors.
- bedrooms: more bedrooms are associated with a higher price, up to 4 bedrooms. After that, it does not look like homes with additional bedrooms are sold at a much higher price.
- condition: values 3, 4, and 5 associated with higher price.
- floors and bedrooms could be treated as categorical in order to better determine the characteristics of each one that are associated with price.

In [ ]:
# for presenation slides

fig, ax = plt.subplots(figsize = (5,6))

sns.boxplot(data = df_pp, x= 'grade', y = 'price')

ax.set(xlabel='Building Grade', ylabel='Home Price ($)')

ax.set_ylim([0, 1600000])
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

sns.despine()

fig.suptitle('Grade', size=13)
fig.subplots_adjust(top=0.9)

In [ ]:
# for presentation slides

fig, ax = plt.subplots(figsize = (5,6))

#order = df_processed.groupby(by=["condition_labels"])["price"].median().sort_values().index

sns.boxplot(data = df_pp, x= 'bathrooms', y = 'price')

ax.set(xlabel='Number of Bathrooms', ylabel='Home Price ($)')

ax.tick_params(axis="x", rotation=90)

ax.set_ylim([0, 1600000])
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

sns.despine()

fig.suptitle('Bathrooms', size=13)
fig.subplots_adjust(top=0.9)

In [ ]:
fig, axes = plt.subplots(3, figsize = (10,10))

#yr_built boxplot
yr_built_boxplot = sns.boxplot(data = df_pp, x= 'yr_built', y = 'price', ax = axes[0])
axes[0].tick_params(axis="x", rotation=90)
# reduce x-tick label frequency to make more readable
axes[0].set_xticks(np.arange(0, 120, 5))
axes[0].set_xticklabels(np.arange(1900, 2020, 5))

#age boxplot
yr_built_boxplot = sns.boxplot(data = df_pp, x= 'age', y = 'price', ax = axes[1])
axes[1].tick_params(axis="x", rotation=90)
# reduce x-tick label frequency to make more readable
axes[1].set_xticks(np.arange(0, 120, 5))
axes[1].set_xticklabels(np.arange(0, 120, 5))

#zipcode boxplot        
zipcode_boxplot = sns.boxplot(data = df_pp, x= 'zipcode', y = 'price', ax = axes[2])
axes[2].tick_params(axis="x", rotation=90)

- yr_built does not have strong linear relationship with price, but it seems that some periods are associated with higher/lower prices. Explore whether treating as categorical variables would help.
- Zipcode also does not have clear linear relationship with price.
- yr_built and zipcode are candidates for one hot encoding. There is no ordering or relationship between the categories and price.

In [ ]:
# Explore age distribution and price by age
df_pp.yr_built.describe()

In [ ]:
mean_price_by_yr_built = df_pp.groupby('yr_built')['price'].mean()
mean_price_by_yr_built.sort_values(ascending=True)

In [ ]:
df_pp.price.mean()

In [ ]:
fig, ax = plt.subplots(figsize = (8,5))

data = df_pp
data['price_mean'] = df_pp.price.mean()

barplot = mean_price_by_yr_built.plot(kind='bar', color='powderblue', label='Mean price by Year Built')
data['price_mean'].plot(kind='line', color = 'navy', label = 'Mean price of Sample: $504,333')

plt.ylabel('Mean Price ($)',size=12)
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])

plt.xlabel('Year Built', size=12)
plt.xticks(rotation=90)
ax.set_xticks(np.arange(0, 120, 5))
ax.set_xticklabels(np.arange(1900,2020,5))

plt.legend(loc='upper center', borderaxespad=0.2, edgecolor='white', fontsize=12)
fig.suptitle('Mean Price by Year Built', fontsize=15)
fig.subplots_adjust(top=0.94)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show();

- Homes built in the early 1900 up to 1930s, and newer homes built from 1990 onwards generally are sold at prices above the sample mean price of $504,333. Homes built from 1940-1990 generally are sold at prices below the sample mean price.

In [ ]:
#Explore zipcode distribution
sns.distplot(df_pp.zipcode)
plt.title('Distribution of Zipcode')

In [ ]:
# Find average price by zipcode
mean_price_by_zip = df_pp.groupby('zipcode')['price'].mean()
mean_price_by_zip.sort_values(ascending=True)

In [ ]:
df_pp.zipcode.mean()

In [ ]:
# Plot average price by zipcode, and add line chart representing sample's mean price.

fig, ax = plt.subplots(figsize = (5,6))
data = df_pp
data['price_mean'] = data.price.mean()

mean_price_by_zip.plot(kind='bar', color='powderblue', label='Mean Price by Zipcode')
data['price_mean'].plot(kind='line', color = 'mediumblue', label = 'Mean Price of Sample: $504,333')

plt.ylabel('Mean Price ($)',size=12)
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])

plt.xlabel('Zipcodes', size=12)
plt.xticks([])

plt.legend(loc='upper right', borderaxespad=0.2, edgecolor='white', fontsize=12)
fig.suptitle('Mean Price: Sample and Zipcodes', fontsize=15)
fig.subplots_adjust(top=0.94)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show();

In [ ]:
# Plot average price by zipcode, and add line chart representing sample's mean price.

fig, ax = plt.subplots(figsize = (15,5))
data = df_pp
data['price_mean'] = data.price.mean()

mean_price_by_zip.plot(kind='bar', color='powderblue', label='Mean Price by Zipcode')
data['price_mean'].plot(kind='line', color = 'mediumblue', label = 'Mean Price of County: $504,333')

plt.ylabel('Mean Price ($)',size=12)
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])

plt.xlabel('Zipcodes', size=12)
plt.xticks(rotation=90)

plt.legend(loc='upper right', borderaxespad=0.2, edgecolor='white', fontsize=12)
fig.suptitle('Mean Price: County and Zipcodes', fontsize=15)
fig.subplots_adjust(top=0.94)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show();

- Zipcode variable does not have normal distribution.
- Mean price varies notably by zipcode. Homes in some select zipcodes have higher mean prices than other zipcodes. There is no order in the relationship of zipcode categories and price.

In [ ]:
# Explore geolocation data (latitute, longitude)
# Scatterplot of longitude (north-south) and latitude (east-west)

data = df_pp.rename(columns={'price': 'Price', 'long': 'Longitute', 'lat': 'Latitude'})
fig = plt.gcf()

fig.set_size_inches(12, 8)

sns.scatterplot(data.Longitute, data.Latitude, hue = data.Price,  \
                          palette="YlOrRd", legend="brief").legend(loc='lower right',   \
                                                                   fontsize = 12, frameon = False)

plt.xlabel('Longitude',size=14)
plt.ylabel('Latitude',size=14)
sns.despine()

fig.suptitle('Price by Location: Latitude and Longitude', fontsize = 15)
fig.subplots_adjust(top=0.95)

In [ ]:
# Number of zipcodes with mean price > sample mean price and number with mean price < sample mean price
price_sample_mean = df_pp.price.mean()
(mean_price_by_zip >= price_sample_mean).value_counts()

- 33 zipcodes have mean price that is greater than the sample's mean price of $504,333.
- 37 zipcodes have mean price that is below sample's mean price.

In [ ]:
df_pp.columns

In [ ]:
ax = sns.boxplot(x="lat", data=df_pp, orient='v')

In [ ]:
sns.pairplot(data=df_pp,
            x_vars=['lat'],
            y_vars=['price'], 
            kind="reg",
            height = 5)

- Location (in this case, determined by latitude) is an important feature of housing prices, but the relationship is not linear.

#### Locate and load city zipcodes.  This will help make zipcode data more interpretable.

- Data on zipcodes by city obtained from King County GIS Open Data 
- https://gis-kingcounty.opendata.arcgis.com/datasets/kingcounty::zipcodes-for-king-county-and-surrounding-area-zipcode-area/about

In [ ]:
# Load zipcodes df. This was obtained from King County GIS Open Data
df_zips = pd.read_csv('data/zipcodes.csv')

In [ ]:
df_zips.info()

In [ ]:
df_zips.head()

In [ ]:
df_zips.ZIPCODE.nunique()

In [ ]:
df_zips.COUNTY_NAME.value_counts()

- The dataset includes zipcodes for other counties. Need to narrow dataset down to only King County.

In [ ]:
# Drop columns that are not needed for zipcode, city joining. Keep zipcode, county name, preferred city.
df_zips.drop(['OBJECTID', 'ZIP', 'COUNTY', 'ZIP_TYPE', 'Shape_Length', 'Shape_Area'], axis=1, inplace=True)

In [ ]:
# Rename columns
column_names = {'ZIPCODE':'zipcode', 'COUNTY_NAME':'county', 'PREFERRED_CITY':'city'}
df_zips.rename(columns=column_names, inplace=True)

In [ ]:
# Drop zipcodes that are not in King County
df_zips = df_zips[df_zips['county']=="King County"]
df_zips.shape

In [ ]:
df_zips.county.value_counts()

In [ ]:
# Convert City values from all caps to title case. Replace space with underscore in County and City values.
df_zips.head()

In [ ]:
df_zips.city = df_zips.city.str.title().replace(" ", "_", regex=True)

In [ ]:
df_zips.county = df_zips.county.str.title().replace(" ", "_", regex=True)

In [ ]:
df_zips

In [ ]:
df_zips.city.value_counts()

In [ ]:
df_zips.city.replace('Sammamiish', 'Sammamish', inplace=True)
df_zips.city.value_counts()

In [ ]:
#Create dictionary with key, value: zipcode, city
zips_dictionary = dict(zip(df_zips.zipcode, df_zips.city))
zips_dictionary

In [ ]:
# Create new column in df_pp that has zipcode values. 
df_pp['city'] = df_pp.zipcode
df_pp['city']

In [ ]:
# Call on dictionary to replace zipcode values in new column with city names.
df_pp.replace({'city': zips_dictionary}, inplace=True)

In [ ]:
df_pp.head()

In [ ]:
df_pp.city.value_counts()

In [ ]:
# Mean price by city
mean_price_by_city = df_pp.groupby('city')['price'].mean()
mean_price_by_city.sort_values(ascending=True)

In [ ]:
# Plot average price by city, and add line chart representing mean price.

fig, ax = plt.subplots(figsize = (8,4))
data = df_pp
data['price_mean'] = data.price.mean()

mean_price_by_city.sort_values(ascending=True).plot(kind='bar', color='powderblue', label='Mean Price by City')
data['price_mean'].plot(kind='line', color = 'mediumblue', label = 'Mean Price of King County: $504,333')

plt.ylabel('Mean Price ($)',size=12)
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])

plt.xlabel('City', size=12)
plt.xticks(rotation=90)
ax.set_ylim([0, 1500000])

plt.legend(loc='upper left', borderaxespad=0.2, edgecolor='white', fontsize=11)
fig.suptitle('Home Prices: County and City', fontsize=15)
fig.subplots_adjust(top=0.94)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show();

In [ ]:
# Create new variables: price per sqft, and mean price per sqft by city
df_pp['price_sqft'] = df_pp.price / df_pp.sqft_above
mean_price_sqft_by_city = df_pp.groupby('city')['price_sqft'].mean()
mean_price_sqft_by_city.sort_values(ascending=True)

In [ ]:
fig, ax = plt.subplots(figsize = (8,4))
data = df_pp

data['price_sqft_mean'] = data.price_sqft.mean()

mean_price_sqft_by_city.sort_values(ascending=True).plot(kind='bar', color='lightgray', label='Mean Price per Sqft by City')
data['price_sqft_mean'].plot(kind='line', color = 'blue', label = 'Mean Price per Sqft of King County: $309')

plt.ylabel('Mean Price per Sqft ($)',size=12)
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])

plt.xlabel('City', size=12)
plt.xticks(rotation=90)
ax.set_ylim([0, 800])

plt.legend(loc='upper left', borderaxespad=0.2, edgecolor='white', fontsize=11)
fig.suptitle('Home Prices per Square Foot: County and City', fontsize=15)
fig.subplots_adjust(top=0.94)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show();


- Five cities have mean price/sqft above the county mean, down from ten cities that have mean price above the county mean.

In [ ]:
data.columns

# Data Transformation
- Create dummy variables
- Check for multicollinearity
- Normalize variables

In [ ]:
df_pp.columns

In [ ]:
# New df for processing. Df is comprised of variables of interest. 
# Drop variables that are unique identifiers and are not home features. Drop yr_built, keep age since these are correlated nearly 1:1.
df_processed = df_pp.drop(['id', 'date', 'view','yr_built','yr_sold', 'quarter_sold','month_sold','day_of_week_sold', 'price_mean', 'view'], axis=1)
df_processed.columns

In [ ]:
# Plot heatmap of correlation matrix, including the target and predictive variables

fig, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(df_processed.corr().round(2),
            ax = ax,
            annot= True, center=0, cmap="YlGnBu",
            cbar_kws={"label": "Correlation", "orientation": "vertical", "pad": .05}
)

ax.set_title("Heatmap of Correlation Between Features (Includes Price Target)");

- Selecting which variables to drop because of multicollinearity (correlation >= .75).
- sqft_living and sqft_above exhibit multicollinearity, with correlation of 86%. Keep sqft_above because it is less highly correlated with other features of interest such as grade and bathrooms, and because most people want to live above ground. Create sqft_basement dummy, and remove sqft_basement (which is component of sqft_living).
- sqft_living and sqft_above are highly correlated iwth sqft_living15. Drop sqft_living15.
- sqft_living and grade have high correlation (74%), below 75% threshold, but both are most correlated with price. Consider treating one hot encoding grade, so can use grade dummies in model along with sqft_living.
- yr_built and age are nearly perfectly inversely correlated. Drop age.

In [ ]:
# Create dummy variable for basement before dropping sqft_basement from features of interest selection
df_processed.sqft_basement.value_counts()

In [ ]:
df_processed['basement'] = df_processed.sqft_basement
df_processed['basement'] = np.where(df_processed.sqft_basement != 0, 1, df_processed['basement'])
df_processed['basement']

In [ ]:
df_processed.basement.value_counts()

In [ ]:
# Variables that could be treated as categoricals and be candidates for dummy variable creation
categoricals = ['floors'],['grade'],['condition'],['bedrooms'],['zipcode'],['city']
categoricals

# other candidates that could be treated as categorical: ['bathrooms']

In [ ]:
# Visualize categorical variables and price
fig, axes = plt.subplots(3,2, sharey=True, figsize=(10,10))

for col, ax in zip(categoricals, axes.flatten()):
    (df_processed.groupby(col)
        .mean()['price']
        .sort_values()
        .plot.bar(ax=ax))
    #ax.set_title(col)
    ax.set_ylabel('price')

fig.tight_layout()    

### Categorical Variables: Dummies and One Hot Encoding

In [ ]:
# create dummy for yr_renovated: y/n
df_processed['renovated'] = np.where(df_processed['yr_renovated']> 0, 1, 0)
df_processed.head()

In [ ]:
df_processed.renovated.value_counts()

In [ ]:
# for project slides

fig, ax = plt.subplots(figsize = (5,7))

ax = sns.boxplot(x="renovated", y='price', data=df_processed, orient='v', palette="Blues")

ax.set(xlabel='Renovated', ylabel='Home Price ($)')

ax.tick_params(axis=("x"), rotation=90)

ax.set_ylim([0, 1600000])
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

sns.despine()

fig.suptitle('Home Has Been Renovated', size=13)
fig.subplots_adjust(top=0.9)

In [ ]:
df_processed.columns

In [ ]:
# Floors variable suggests that price drops if there too many floors.
# ohe for levels (one level, two levels, three-plus levels). Half floors round up (eg, 2.5 floors = 3 levels)
# Get sense of if people pay more or less for single level vs. multi-level homes. Do people prefer single or multi-story?

bins = pd.IntervalIndex.from_tuples([(0,1), (1,2), (2, 3.5)])

levels_bins = pd.cut(df_processed['floors'], bins)

levels_bins.categories = ['levels_single','levels_two', 'levels_threeplus']

levels_dummies = pd.get_dummies(levels_bins.cat.rename_categories(levels_bins.categories), drop_first=True)

levels_dummies

In [ ]:
levels_dummies.levels_two.value_counts()

In [ ]:
# ohe for number of floors
bins = pd.IntervalIndex.from_tuples([(0,1), (1,1.5), (1.5,2), (2,2.5), (2.5,3), (3,3.5)])

fl_bins = pd.cut(df_pp['floors'], bins)

fl_bins.categories = ['floors_1','floors_1.5', 'floors_2','floors_2.5', 'floors_3', 'floors_3.5']

fl_number_dummies = pd.get_dummies(fl_bins.cat.rename_categories(fl_bins.categories), drop_first=True)

fl_number_dummies

In [ ]:
levels_dummies_test = pd.concat([df_processed.price, df_processed.floors, levels_dummies, fl_number_dummies], axis=1)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,5))
sns.boxplot(data=df_processed, x='floors', y='price', ax = ax[0])
sns.boxplot(data=levels_dummies_test, x='levels_two', y='price', ax = ax[1])
sns.boxplot(data=levels_dummies_test, x='levels_threeplus', y='price', ax = ax[2])

In [ ]:
levels_dummies_test.corr().round(2)

- Select levels_two and levels_threeplus for model. Levels_two has same price correlation as floors-price correlation (27%), but add more granular insights.

In [ ]:
# ohe grade
grade_cat = 'grade'
grade_dummies = pd.get_dummies(df_processed.grade, prefix=grade_cat, drop_first=True)
grade_dummies

In [ ]:
grade_dummies_test_df = pd.concat([df_processed.price, df_processed.grade, grade_dummies], axis=1)

In [ ]:
grade_dummies_test_df.corr()

- grades 6, 7, 9, 10, 11, 12 have strongest correlations with price. Grade variable is not strongly correlated with the grade dummy variables.

In [ ]:
# Prepare condition for ohe
df_processed.condition.value_counts()

In [ ]:
pd.cut(df_processed['condition'], [0,2.9, 3, 5]).value_counts()

In [ ]:
# Label encode categorical dummies
cond_labels = {1: "poor", 2: "fair", 3: "average", 4: "good", 5: "very_good"}
df_processed['condition_labels'] = df_processed.condition
df_processed.replace({'condition_labels': cond_labels}, inplace=True)

In [ ]:
# ohe condition 
condition_cat = 'condition'
condition_dummies = pd.get_dummies(df_processed.condition_labels, prefix=condition_cat)
condition_dummies.drop('condition_poor', axis=1, inplace=True)

In [ ]:
df_pp.condition.value_counts()

In [ ]:
df_processed.condition_labels.value_counts()

In [ ]:
df_processed.groupby(by=["condition_labels"])["price"].median().sort_values().index

In [ ]:
# chart for presentation slides

fig, ax = plt.subplots(figsize = (5,6))

order = df_processed.groupby(by=["condition_labels"])["price"].median().sort_values().index

sns.boxplot(data = df_processed, x= 'condition_labels', y = 'price', order=order)

ax.set(xlabel='Building Condition', ylabel='Home Price ($)')

ax.set_ylim([0, 1600000])
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

sns.despine()

fig.suptitle('Condition', size=13)
fig.subplots_adjust(top=0.9)

In [ ]:
sns.boxplot(data = df_pp, x= 'condition', y = 'price')

In [ ]:
condition_dummies_test_df = pd.concat([condition_dummies, df_processed['condition'], df_processed['price']], axis=1)
                                  

In [ ]:
#Examine relationships between condition, new dummy variables for condition, and price
condition_dummies_test_df.corr().round(2)

In [ ]:
test_data = condition_dummies_test_df
fig, axes = plt.subplots(2,2, figsize=(8,6))
fig.suptitle('Price by Condition')
sns.boxplot(ax=axes[0,0], data=test_data, x='condition_fair', y='price')
sns.boxplot(ax=axes[0,1], data=test_data, x='condition_average', y='price')
sns.boxplot(ax=axes[1,0], data=test_data, x='condition_good', y='price')
sns.boxplot(ax=axes[1,1], data=test_data, x='condition_very_good', y='price')
plt.tight_layout()

- Condition: no clear ordering of condition values and higher price. 
- Looks like values of average, good, and very good are associated with higher price, with not much difference between them. Values of poor and fair are associated with lower price.

In [ ]:
# ohe bedrooms
bedrooms_cat = 'bedrooms'
bedrooms_dummies = pd.get_dummies(df_processed.bedrooms, prefix=bedrooms_cat, drop_first=True)
bedrooms_dummies

In [ ]:
bedrooms_dummies_test_df = pd.concat([df_processed.price, df_processed.bedrooms, bedrooms_dummies], axis=1)

In [ ]:
bedrooms_dummies_test_df.corr().round(2)

In [ ]:
# ohe bathrooms
bathrooms_cat = 'bathrooms'
bathrooms_dummies = pd.get_dummies(df_pp.bathrooms, prefix=bathrooms_cat, drop_first=True)
bathrooms_dummies

In [ ]:
bathrooms_dummies_test_df = pd.concat([df_pp.price, df_pp.bathrooms, bathrooms_dummies], axis=1)

In [ ]:
bathrooms_dummies_test_df.corr()

In [ ]:
# ohe city
city_dummies = pd.get_dummies(df_pp.city, prefix= 'city', drop_first=False)
city_dummies

In [ ]:
#Since did not drop_first in ohe above, drop "city_Seattle" to make Seattle the reference city in the city dummy variable interpretation
city_dummies.drop(['city_Seattle'], axis=1, inplace=True)

In [ ]:
city_dummies_test_df = pd.concat([df_pp.price, city_dummies], axis=1)
city_dummies_test_df.corr()

In [ ]:
df_processed.columns

In [ ]:
df_processed = pd.concat([df_processed, levels_dummies, fl_number_dummies, grade_dummies, condition_dummies,
                         bedrooms_dummies, bathrooms_dummies, city_dummies], axis=1)
df_processed.shape

In [ ]:
df_processed.columns

In [ ]:
# Create dummies list
dummies = ['levels_two', 'levels_threeplus', 'floors_1.5', 'floors_2',
       'floors_2.5', 'floors_3', 'floors_3.5','grade_4', 'grade_5', 'grade_6', 'grade_7',
       'grade_8', 'grade_9', 'grade_10', 'grade_11', 'grade_12', 'condition_2',
       'condition_3', 'condition_4', 'condition_5', 'condition_average', 'condition_above',
        'bedrooms_2', 'bedrooms_3', 'bedrooms_4', 'bedrooms_5', 'bedrooms_6', 
        'bathrooms_0.75', 'bathrooms_1.0', 'bathrooms_1.25', 'bathrooms_1.5',
       'bathrooms_1.75', 'bathrooms_2.0', 'bathrooms_2.25', 'bathrooms_2.5',
       'bathrooms_2.75', 'bathrooms_3.0', 'bathrooms_3.25', 'bathrooms_3.5',
       'bathrooms_3.75', 'bathrooms_4.0', 'bathrooms_4.25', 
        'city_Auburn', 'city_Bellevue', 'city_Black_Diamond', 'city_Bothell',
       'city_Carnation', 'city_Duvall', 'city_Enumclaw', 'city_Fall_City',
       'city_Federal_Way', 'city_Issaquah', 'city_Kenmore', 'city_Kent',
       'city_Kirkland', 'city_Maple_Valley', 'city_Medina',
       'city_Mercer_Island', 'city_North_Bend', 'city_Redmond', 'city_Renton',
       'city_Sammamish', 'city_Seattle', 'city_Snoqualmie', 'city_Vashon',
       'city_Woodinville']

In [ ]:
# Re-examine correlations with price, and multicollinearity of variables (including new dummy variables)
# Multicollinearity of variables
data_test = df_processed

df=data_test.corr().round(2).abs().stack().reset_index().sort_values(0, ascending=False)

# zip the variable name columns (Which were only named level_0 and level_1 by default) in a new column named "pairs"
df['pairs'] = list(zip(df.level_0, df.level_1))

# set index to pairs
df.set_index(['pairs'], inplace = True)

#drop level columns
df.drop(columns=['level_1', 'level_0'], inplace = True)

# rename correlation column as cc rather than 0
df.columns = ['cc']

df.drop_duplicates(inplace=True)

In [ ]:
df[(df.cc>.6) & (df.cc <1)].abs()

- list returns many highly correlated pairs, since it includes continuous variables and their dummy variables. Linear regression models (below) will not include such correlated variable-dummy pairs.
- grade and sqft_living are most highly correlated with price and should be part of baseline model. However, these two variables are also highly correlated with eachother. Use grade dummies in model, in place of grade.
- bathrooms has next highest correlation with price, but it's lower at 49%.

In [ ]:
# Plot pairplots for price and grade, sqft_above
pp = sns.pairplot(data=df_pp,
            x_vars=['grade', 'sqft_above'],
            y_vars=['price'], 
            kind="reg",
            height = 5)
pp.fig.suptitle('Baseline Model Features')

plt.show()

## MODELING

### Baseline Model

- Build Baseline model:
    - This baseline model includes sqft_above and grade, which are two independent variables with highest correlations with price. Because sqft_above and grade are correlated somewhat highly at 73.5%, this model includes grade dummy variables and not the continuous grade variable.
    - Variables are not logged nor scaled at this point.

In [ ]:
data = df_processed
X_data = data.drop(['price'], axis=1)

In [ ]:
y = data.price 
X_model_baseline = data[['sqft_above', 'grade_9','grade_10','grade_11', 'grade_12']] 

In [ ]:
X_model_baseline = sm.add_constant(X_model_baseline)

In [ ]:
model_baseline = sm.OLS(y, X_model_baseline).fit()
model_baseline.summary()

- r-squared: baseline model explains 41.7% of variance in dependent variable, price. There is room to improve this model through obtaining a higher r-squared in subsequent model iterations. Follow along for more!
- sqft_above and grade_9 - grade_12 are statistically significant. The other grade dummy variables (grade_4 - grade_8) are not statistically different from the dropped grade_3, so these were excluded from the baseline model.
- Coefficient signs are positive. 
    - For every unit increase in sqft_above, price increases by USD 91.7. 
    - A home with a given grade_9, grade_10, grade_11, and grade_12 would be worth more than a home with grades_4-grade_8 by USD 213.9k, USD 353k, USD 490k, and USD 711k, respectively.

### Selecting Best Combination of Features

In [ ]:
#Model 2: Add structural features: bathrooms, floor dummies, basement. 
y = data.price 
X_model_2 = data[['sqft_above', 'grade_9','grade_10','grade_11', 'grade_12',
                  'bathrooms', 'levels_two', 'levels_threeplus', 'basement']]

In [ ]:
model_2 = sm.OLS(y, sm.add_constant(X_model_2)).fit()
model_2.summary()

In [ ]:
#Model 3: Add selling features: waterfront, sqft_lot, condition, renovated.
y = data.price 
X_model_3 = data[['sqft_above', 'grade_9','grade_10','grade_11', 'grade_12',
                  'bathrooms', 'levels_two', 'levels_threeplus', 'basement',
                 'waterfront', 'sqft_lot','condition', 'renovated']]

In [ ]:
model_3 = sm.OLS(y, sm.add_constant(X_model_3)).fit()
model_3.summary()

In [ ]:
#Model 4: Add location-related variables--sqft_lot15, city dummy variables.

y = data.price 
X_model_4 = data[['sqft_above', 'grade_9','grade_10','grade_11', 'grade_12',
                  'bathrooms', 'levels_two', 'levels_threeplus', 'basement',
                 'waterfront', 'sqft_lot','condition', 'renovated','sqft_lot15',
                  'city_Auburn', 'city_Bellevue', 'city_Black_Diamond', 'city_Bothell', 
                  'city_Carnation', 'city_Duvall', 'city_Enumclaw', 'city_Fall_City', 
                  'city_Federal_Way', 'city_Issaquah', 'city_Kenmore', 'city_Kent', 
                  'city_Kirkland', 'city_Maple_Valley', 'city_Medina', 'city_Mercer_Island', 
                  'city_North_Bend', 'city_Redmond', 'city_Renton', 'city_Sammamish', 
                  'city_Snoqualmie', 'city_Vashon', 'city_Woodinville']]

In [ ]:
model_4 = sm.OLS(y, sm.add_constant(X_model_4)).fit()
model_4.summary()

In [ ]:
# Model 5: Drop non-statistically significant variables: sqft_lot15, city_Redmond. Drop condition to reduce multicollinearity.
y = data.price 
X_model_5 = data[['sqft_above', 'grade_9','grade_10','grade_11', 'grade_12',
                  'bathrooms', 'levels_two', 'basement',
                 'waterfront', 'sqft_lot', 'renovated',
                  'city_Auburn', 'city_Bellevue', 'city_Black_Diamond', 'city_Bothell', 
                  'city_Duvall', 'city_Enumclaw', 'city_Fall_City', 
                  'city_Federal_Way', 'city_Issaquah', 'city_Kenmore', 'city_Kent', 
                  'city_Kirkland', 'city_Maple_Valley', 'city_Medina', 'city_Mercer_Island', 
                  'city_North_Bend', 'city_Renton', 'city_Sammamish', 
                  'city_Snoqualmie', 'city_Vashon', 'city_Woodinville']]

In [ ]:
model_5 = sm.OLS(y, sm.add_constant(X_model_5)).fit()
model_5.summary()

In [ ]:
# Model 6: Replace bathrooms with bathroom dummies
y = data.price 
X_model_6 = data[['sqft_above', 'grade_9','grade_10','grade_11', 'grade_12',
                  'bathrooms_1.0','bathrooms_1.25', 'bathrooms_1.5',
                  'bathrooms_1.75', 'bathrooms_2.0', 'bathrooms_2.25', 'bathrooms_2.5',
                  'bathrooms_2.75', 'bathrooms_3.0', 'bathrooms_3.25', 'bathrooms_3.5',
                  'bathrooms_3.75', 'bathrooms_4.0', 'bathrooms_4.25', 
                  'levels_two', 'basement',
                 'waterfront', 'sqft_lot','renovated',
                  'city_Auburn', 'city_Bellevue', 'city_Black_Diamond', 'city_Bothell', 
                  'city_Duvall', 'city_Enumclaw', 'city_Fall_City', 
                  'city_Federal_Way', 'city_Issaquah', 'city_Kenmore', 'city_Kent', 
                  'city_Kirkland', 'city_Maple_Valley', 'city_Medina', 'city_Mercer_Island', 
                  'city_North_Bend', 'city_Renton', 'city_Sammamish', 
                  'city_Snoqualmie', 'city_Vashon', 'city_Woodinville']]

In [ ]:
model_6 = sm.OLS(y, sm.add_constant(X_model_6)).fit()
model_6.summary()

In [ ]:
# Model 7: Drop bathroom dummies that are not statistically significant (bathrooms_1.0). Add statistically significant condition_good and condition_very_good.
y = data.price 
X_model_7 = data[['sqft_above', 'grade_8','grade_9','grade_10','grade_11', 'grade_12',
                  'bathrooms_1.25', 'bathrooms_1.5', 'bathrooms_1.75', 'bathrooms_2.0', 'bathrooms_2.25', 'bathrooms_2.5',
                  'bathrooms_2.75', 'bathrooms_3.0', 'bathrooms_3.25', 'bathrooms_3.5',
                  'bathrooms_3.75', 'bathrooms_4.0', 'bathrooms_4.25', 
                  'levels_two', 'basement',
                  'waterfront', 'sqft_lot', 'renovated', 'condition_good', 'condition_very_good',
                  'city_Auburn', 'city_Bellevue', 'city_Black_Diamond', 'city_Bothell', 
                  'city_Duvall', 'city_Enumclaw', 'city_Fall_City', 
                  'city_Federal_Way', 'city_Issaquah', 'city_Kenmore', 'city_Kent', 
                  'city_Kirkland', 'city_Maple_Valley', 'city_Medina', 'city_Mercer_Island', 
                  'city_North_Bend', 'city_Renton', 'city_Sammamish', 
                  'city_Snoqualmie', 'city_Vashon', 'city_Woodinville']]

In [ ]:
model_7 = sm.OLS(y, sm.add_constant(X_model_7)).fit()
model_7.summary()

In [ ]:
# Model 8: Drop bathrooms_1.25, bathrooms_1.5, bathrooms_2.5, which are statistically insignificant.
y = data.price 
X_model_8 = data[['sqft_above', 'grade_8','grade_9','grade_10','grade_11', 'grade_12',
                  'bathrooms_1.75', 'bathrooms_2.0', 'bathrooms_2.25',
                  'bathrooms_2.75', 'bathrooms_3.0', 'bathrooms_3.25', 'bathrooms_3.5',
                  'bathrooms_3.75', 'bathrooms_4.0', 'bathrooms_4.25', 
                  'levels_two', 'basement',
                  'waterfront', 'sqft_lot', 'renovated', 'condition_good', 'condition_very_good',
                  'city_Auburn', 'city_Bellevue', 'city_Black_Diamond', 'city_Bothell', 
                  'city_Duvall', 'city_Enumclaw', 'city_Fall_City', 
                  'city_Federal_Way', 'city_Issaquah', 'city_Kenmore', 'city_Kent', 
                  'city_Kirkland', 'city_Maple_Valley', 'city_Medina', 'city_Mercer_Island', 
                  'city_North_Bend', 'city_Renton', 'city_Sammamish', 
                  'city_Snoqualmie', 'city_Vashon', 'city_Woodinville']]

In [ ]:
model_8 = sm.OLS(y, sm.add_constant(X_model_8)).fit()
model_8.summary()

In [ ]:
# Model 9: Drop bathrooms_2.25, which is statistically insignificant.
y = data.price 
X_model_9 = data[['sqft_above', 'grade_8','grade_9','grade_10','grade_11', 'grade_12',
                  'bathrooms_1.75', 'bathrooms_2.0', 
                  'bathrooms_2.75', 'bathrooms_3.0', 'bathrooms_3.25', 'bathrooms_3.5',
                  'bathrooms_3.75', 'bathrooms_4.0', 'bathrooms_4.25',
                  'levels_two', 'levels_threeplus', 'basement',
                  'waterfront', 'sqft_lot', 'renovated', 'condition_good', 'condition_very_good',
                  'city_Auburn', 'city_Bellevue', 'city_Black_Diamond', 'city_Bothell', 
                  'city_Duvall', 'city_Enumclaw', 'city_Fall_City', 
                  'city_Federal_Way', 'city_Issaquah', 'city_Kenmore', 'city_Kent', 
                  'city_Kirkland', 'city_Maple_Valley', 'city_Medina', 'city_Mercer_Island', 
                  'city_North_Bend', 'city_Renton', 'city_Sammamish', 
                  'city_Snoqualmie', 'city_Vashon', 'city_Woodinville']]

In [ ]:
X_model_9 = sm.add_constant(X_model_9)

In [ ]:
model_9 = sm.OLS(y, X_model_9).fit()
model_9.summary()

#### Log transformation

In [ ]:
data['price_log'] = np.log(data.price)
data['sqft_above_log'] = np.log(data.sqft_above)

In [ ]:
#Model 10:  Log transform price. Use independent variables in Model9.
y_log = data.price_log
X_model_10 = data[['sqft_above', 'grade_8','grade_9','grade_10','grade_11', 'grade_12',
                  'bathrooms_1.75', 'bathrooms_2.0', 
                  'bathrooms_2.75', 'bathrooms_3.0', 'bathrooms_3.25', 'bathrooms_3.5',
                  'bathrooms_3.75', 'bathrooms_4.0', 'bathrooms_4.25', 
                  'levels_two', 'basement',
                  'waterfront', 'sqft_lot', 'renovated', 'condition_good', 'condition_very_good',
                  'city_Auburn', 'city_Bellevue', 'city_Black_Diamond', 'city_Bothell', 
                  'city_Duvall', 'city_Enumclaw', 'city_Fall_City', 
                  'city_Federal_Way', 'city_Issaquah', 'city_Kenmore', 'city_Kent', 
                  'city_Kirkland', 'city_Maple_Valley', 'city_Medina', 'city_Mercer_Island', 
                  'city_North_Bend', 'city_Renton', 'city_Sammamish', 
                  'city_Snoqualmie', 'city_Vashon', 'city_Woodinville']]


In [ ]:
X_model_10 = sm.add_constant(X_model_10)

In [ ]:
model_10 = sm.OLS(y, X_model_10).fit()
model_10.summary()

- When log price is dependent variable, dependent variable sqft_above's confidence interval is reduced to zero. This suggests that the variable may not be different from the null hypothesis of sqft_above=0. In other words, the null hypothesis that sqft_above = 0 cannot be rejected. This variable is not significant to the model. In next iteration, use log sqft_above.

In [ ]:
#Model 11: Drop statistically insignificant bathroom dummy variables: bathrooms_3.5, bathrooms_4.0,
# bathrooms_3.75, bathrooms_4.0, bathrooms_4.25. Replace sqft_above with log sqft_above.

y_log = data.price_log
X_model_11 = data[['sqft_above_log', 'grade_8','grade_9','grade_10','grade_11', 'grade_12',
                  'bathrooms_1.75', 'bathrooms_2.0', 
                  'bathrooms_2.75', 'bathrooms_3.0', 
                  'levels_two', 'basement',
                  'waterfront', 'sqft_lot', 'renovated', 'condition_good', 'condition_very_good',
                  'city_Auburn', 'city_Bellevue', 'city_Black_Diamond', 'city_Bothell', 
                  'city_Duvall', 'city_Enumclaw', 'city_Fall_City', 
                  'city_Federal_Way', 'city_Issaquah', 'city_Kenmore', 'city_Kent', 
                  'city_Kirkland', 'city_Maple_Valley', 'city_Medina', 'city_Mercer_Island', 
                  'city_North_Bend', 'city_Renton', 'city_Sammamish', 
                  'city_Snoqualmie', 'city_Vashon', 'city_Woodinville']]

In [ ]:
X_model_11 = sm.add_constant(X_model_11)

In [ ]:
model_11 = sm.OLS(y_log, X_model_11).fit()
model_11.summary()

### Selected Model: 
Model 9 has same r-squared (when accounting for rounding) and features as models 10 and 11, which include log transformations. Select model 9 for ease of interpretability in this analysis that is meant to be shared with the broader public in King County. However, testing assumptions (below) will be important to determine whether to move forward with this model, another model, and to continue iterating on model improvements.

In [ ]:
model_9 = sm.OLS(y, X_model_9).fit()
model_9.summary()

### Model Performance

In [ ]:
#Get predictions
y_pred = model_9.predict()

In [ ]:
# MAE - Mean Absolute Error
mean_absolute_error(y, y_pred)

In [ ]:
# MSE - Mean Squared Error
mean_squared_error(y, y_pred)

In [ ]:
# RMSE - Root Mean Squared Error
np.sqrt(mean_squared_error(y, y_pred))

- High MAE, MSE, and RMSE indicate that the model is not accurate in predicting house prices.
- However, the r-squared of 71% indicates that the model does generally well (with more room for improvement) in explaining the variation in house prices.

### Contextual calculations to support interpretation of model coefficients

In [ ]:
print(data.loc[df_processed.grade == 12, ['price']].mean())
print(data.loc[df_processed.grade == 8, ['price']].mean())

In [ ]:
data.grade.mean().round(2)

In [ ]:
print(data.loc[df_processed.bathrooms == 3, ['price']].mean())

### Model Interpretation: Model 9
The model can explain 71% of the variance in house prices in King County, Washington. It takes into account a range of features, including: 
- House structure: square footage, bathrooms, levels, basement
- House quality: grade of construction, condition, renovations 
- House location: lot size, waterfront, and location as determined by City


Coefficient interpretation:
- Construction grade: The largest price increases are associated with a higher grade of construction. Homes with custom design, excellent builders, and of the highest quality (grade 12) are worth USD 540k more than average and below average grade homes. Even homes that have just above average construction (grade 8) are worth USD 85k more than homes with average and below average construction grades. For reference, the average price of grade 12 homes is USD 1.39M, and the average price of grade 8 homes is USD 538k, which is higher than the county mean home price of USD 504k. The average grade of homes in King County is 7.6, and the median grade is 7.0.


- Bathrooms: A higher number of bathrooms is associated with higher prices. Compared to three-quarters of a bathroom, one full-sized bathroom, and one full bathroom plus a second bathroom with only a toilet (1.25 bath), homes with one and three-quarter bathrooms are worth USD 10.6k more, homes with two full-sized bathrooms are worth USD 13.9k more, homes with three full-sized bathrooms are worth USD 25.1k more, and homes with four full-sized bathrooms are worth USD 59.8k more. The average number of bathrooms in King County homes is two, and the median is 2.25.


- Levels: Homes with two levels above ground are associated with a USD 15k higher price and three-plus story homes are associated with USD 28k higher price compared to single-story homes. The median number of floors in King County homes is one, and the median is just shy of 1.5 floors.


- Basement: Homes with a basement are worth USD 86k more than homes without a basement.


- Waterfront: Homes located on the waterfront are worth USD 331k more than homes not located on the waterfront.


- Renovated: Homes that have been renovated are worth USD 92k more than homes that have not been renovated. The average price of renovated homes in King County is USD 642.6k.


- Condition: Homes in better condition are associated with higher prices. Compared to poor, fair, and average conditions, homes in good condition are worth USD 47k more. Homes in very good condition are worth USD 103k more.


- Square footage above ground: A unit increase in square footage is associated with a USD 115 increase in home price.


- Square footage of lot: A unit increase in square footage is associated with a smaller USD 0.17 increase in home price.


- Location: Homes' location matters a lot. Compared to Seattle and Redmond, homes in other King County cities are worth up to USD 598k more and USD 83k less depending on which city they are in.


Other insights:

- Interestingly, the model does not include bedrooms, which was determined to not be statistically significant in the model. Holding all other variables constant, more bedrooms are not associated with higher home prices. For example, this could be because home owners may find it unappealing to have more rooms if square footage does not increase and rooms are small, if there are not more bathrooms, and if the house is not situated in their preferred location.

- Further work could assess location with more granularity as well as incorporate age-related features and analyze the relationship between price and the bedrooms-to-bathrooms ratio.

### Testing Assumptions

#### Linearity

In [ ]:
# Baseline model
preds = model_baseline.predict()
fig, ax = plt.subplots()

perfect_line = np.arange(data.price.min(), data.price.max())
ax.plot(perfect_line, linestyle="--", color="orange", label="Perfect Fit")
ax.scatter(data.price, preds, alpha=0.5)
ax.set_xlabel("Actual Price")
ax.set_ylabel("Predicted Price")
ax.legend();

In [ ]:
# Selected model: Model 9
fig, ax = plt.subplots(figsize = (5,6))

preds = model_9.predict()

perfect_line = np.arange(data.price.min(), data.price.max())
ax.plot(perfect_line, linestyle="--", color="orange", label="Perfect Fit")
ax.scatter(data.price, preds, alpha=0.5)
ax.set_xlabel("Actual Price")
ax.set_ylabel("Predicted Price")
ax.legend();

plt.xticks(rotation=90)
ax.xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
sns.despine()


- Selected model (model 9) has outliers but there is an overall linear relationship. The linearity assumption is not violated.

#### Normality

In [ ]:
# Test the normality assumption with QQ plot
residuals = model_baseline.resid
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
plt.show()

In [ ]:
residuals = model_9.resid  
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
plt.show()

In [ ]:
residuals = model_11.resid   #log price and log sqft_above
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
plt.show()

- Outliers are problematic in the baseline and model 9, and look worse in model 9 versus the baseline model in terms of the normality assumption. 
- Including log price and log sqft_above in the model (instead of price and sqft_above) improves the normality assumption. This suggests that model 11 is better than model 9 in regards to the normality assumption.

#### Multicollinearity

In [ ]:
# Baseline model
vif = [variance_inflation_factor(X_model_baseline.values, i) for i in range(X_model_baseline.shape[1])]
pd.Series(vif, index=X_model_baseline.columns, name="Variance Inflation Factor")

In [ ]:
# Selected model
vif = [variance_inflation_factor(X_model_9.values, i) for i in range(X_model_9.shape[1])]
pd.Series(vif, index=X_model_9.columns, name="Variance Inflation Factor")

- Low mutlicollinearity in baseline model and selected model 9. Model 11 also has low multicolllinearity.

#### Homoscedasticity

In [ ]:
# Baseline model
fig, ax = plt.subplots()

preds = model_baseline.predict()
ax.scatter(preds, residuals, alpha=0.5)
ax.plot(preds, [0 for i in range(len(X_model_7))])
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");

In [ ]:
# Selected model: Model 9
fig, ax = plt.subplots()

preds = model_9.predict()
ax.scatter(preds, residuals, alpha=0.5)
ax.plot(preds, [0 for i in range(len(X_model_7))])
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");

In [ ]:
# Selected model: Model 11  #log price and log sqft_above
fig, ax = plt.subplots()

preds = model_11.predict()
ax.scatter(preds, residuals, alpha=0.5)
ax.plot(preds, [0 for i in range(len(X_model_8))])
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");

- Residuals in selected model 9 (and model 11) are heteroscedastic, varying based on the predicted price. This is a result of how outliers were removed in the preprocessing. Recall that--for price, bedrooms, and bathrooms--maximum thresholds were set at three standard deviations above the mean. Values above these thresholds were dropped.
- Homoskedasticity assumption is violated.